## ここはいろいろ実験的なコードを書くところとする。

## 6/29 submit1.tsvで使ったRandomForestでは、変数の重要度はどうだったのだろうか
今保留になっている号棟データの接道方位や現場データの道路の方位が上位なら、やはり実装が必要と言うことになる

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
model = joblib.load('data/randomforest_r1.model')
x_train = pd.read_csv('data/processed_train_goto_x.csv')

In [7]:
x_train = x_train.drop(['id','pj_no'],axis=1)
importance = pd.DataFrame(model.feature_importances_, index=x_train.columns)
importance.to_csv("data/importance_submit1.csv")

⇒importance_submit1にコメント記入した。

課題で使うことになっている評価関数(Mean Absolute Percentage Error)をGridSearchCVで使うため、評価関数を自作

In [8]:
import numpy as np
def mean_absolute_percentage_error( y_train: np.array, y_pred: np.array):
    diff = 0
    n = len(y_train)
    for i in range(n):
        diff += abs(y_train[i]-y_pred[i])/y_train[i]
    score = 100*diff / n
    
    return score

In [9]:
t = np.array([100,100,100,100,100])
p = np.array([90,110,100,100,100])

In [10]:
print (mean_absolute_percentage_error(t,p))

4.0


trainとtestのデータから、同じダミー変数列を生成するためのコード

In [20]:
def create_list( df1, df2, cols ):
    names = set()
    for col in cols:
        for i in range(len(df1)):
            name = df1.at[i,col]
            if ( type(name) != float ):
                names.add(name)
        for i in range(len(df2)):
            name = df2.at[i,col]
            if ( type(name) != float ):
                names.add(name)
    return sorted(list(names))

In [21]:
import pandas as pd
import numpy as np
train = pd.read_csv("data/train_genba.tsv",sep='\t')
test = pd.read_csv("data/test_genba.tsv",sep='\t')
hokakisei_list = create_list(train, test, ['hokakisei1','hokakisei2','hokakisei3','hokakisei4'])

In [22]:
print(hokakisei_list)

['43条第1項ただし書き許可', '43条許可', '55条許可', '下水道法', '公拡法', '公有地拡大推進法', '区画整理法', '国土法', '土地区画整理法第７６条', '土壌汚染対策法', '埋蔵文化財', '外壁後退', '安行出羽地区地区計画', '小規模宅地開発', '建築協定', '文化財保護法', '文化財保護法（埋蔵文化財）', '景観地区', '景観法', '東日本大震災復興特別区域法', '東日本震災復興特', '河川法', '特定空港周辺特別措置法', '特定都市河川浸水', '生産緑地法', '自然公園法', '航空法', '農地法', '農地法届出要', '都市公園法', '都市再生特別措置法', '風致地区', '高さ最高限度有', '７６条申請']


In [24]:
train = pd.read_csv("data/train_goto.tsv",sep='\t')
test = pd.read_csv("data/train_goto.tsv",sep='\t')
kobetsu_list = create_list( train, test, ['kobetsu1','kobetsu2','kobetsu3','kobetsu4'])
print(kobetsu_list)

['ごみ置き場前', 'アパート南隣', 'エネファーム付', '三方路', '二方路', '交通量多い', '信号前', '信号近い', '前面道が坂途中', '地役権有', '嫌悪施設隣接', '宅内高低差あり', '床暖房付', '心理的瑕疵あり', '敷延2ｍ絞りあり', '横断歩道前', '眺望良', '行き止まり', '行き止まり途中', '街道沿い', '裏道', '角地', '計画道路', '踏切付近', '車進入困難', '高圧線下']


### 現場データのhokakisei1～4から出現するデータをリストにして返す

### 号棟データのkobetsu1～4をダミー変数化する

In [1]:
def put_name(col, name):
    if ( type(name) != float ):
        col.add(name)

import numpy as np
import pandas as pd
import math
train_goto = pd.read_csv("data/train_goto.tsv",sep='\t')
df = train_goto.copy()

cols = set()
for i in range(len(df)):
    put_name(cols, df.at[i,'kobetsu1'])
    put_name(cols, df.at[i,'kobetsu2'])
    put_name(cols, df.at[i,'kobetsu3'])
    put_name(cols, df.at[i,'kobetsu4'])
    
kobetsu_map = {'信号近い':'信号前'}
df['kobetsu1']=df['kobetsu1'].replace(kobetsu_map)
df['kobetsu2']=df['kobetsu2'].replace(kobetsu_map)
df['kobetsu3']=df['kobetsu3'].replace(kobetsu_map)
df['kobetsu4']=df['kobetsu4'].replace(kobetsu_map)

# ステップ２：
kobetsu_dict = {}
for x in cols:
    kobetsu_dict[x] = np.zeros(len(df), dtype=np.int)

# ステップ３：
for i in range(len(df)):
    name = df.at[i,'kobetsu1']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1
    name = df.at[i,'kobetsu2']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1
    name = df.at[i,'kobetsu3']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1
    name = df.at[i,'kobetsu4']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1

# 元のDataFrameとダミー変数化したものを連結
new_genba = pd.concat([df, pd.DataFrame(kobetsu_dict)], axis=1)
new_genba.to_csv("data/temp.csv")

# 不要になったkobetsu1～kobestu4を削除


In [40]:
cols

{'ごみ置き場前',
 'アパート南隣',
 'エネファーム付',
 '三方路',
 '二方路',
 '交通量多い',
 '信号前',
 '信号近い',
 '前面道が坂途中',
 '地役権有',
 '嫌悪施設隣接',
 '宅内高低差あり',
 '床暖房付',
 '心理的瑕疵あり',
 '敷延2ｍ絞りあり',
 '横断歩道前',
 '眺望良',
 '行き止まり',
 '行き止まり途中',
 '街道沿い',
 '裏道',
 '角地',
 '計画道路',
 '踏切付近',
 '車進入困難',
 '高圧線下'}

### RandomForestを使って特徴量の重要度を求める

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
x_train = pd.read_csv('data/processed_train_goto.csv')
Y_train = pd.DataFrame(x_train['tc_tanka'])
X_train = x_train.drop(['id','pj_no','tc_tanka'],axis=1)

In [3]:
model = RandomForestRegressor()
model.fit(X_train.values, Y_train.values.ravel())

C:\Users\satos\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [4]:
print(X_train.columns)

Index(['tc_mseki', 'tt_mseki', 'fukuin', 'road_st', 'magutchi', 'niwasaki',
       'garage', 'chiseki_js_hb', 'chiseki_kb_hb', 'josui',
       ...
       '信号近い', '踏切付近', '地役権有', '行き止まり途中', '嫌悪施設隣接', '眺望良', '敷延2ｍ絞りあり',
       'ごみ置き場前', '前面道が坂途中', 'エネファーム付'],
      dtype='object', length=261)


In [5]:
importance = pd.DataFrame(model.feature_importances_.reshape(1,len(model.feature_importances_)), columns=X_train.columns)
importance.to_csv("data/importance1.csv")
# ⇒　横に長い一覧表になる

In [6]:
importance = pd.DataFrame(model.feature_importances_, index=X_train.columns)
importance.to_csv("data/importance2.csv")
# ⇒縦に長い一覧表になる

### 四つの列からなる法規制列のダミー変数化を行う
ダミー変数化の前には、同一だが違う名前のついているデータを一意にする


In [1]:
import numpy as np
import pandas as pd
import math
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')

#### 統一が必要なカラム
- 公拡法　と　公有地拡大推進法　⇒　公拡法へ
- 埋蔵文化財　と　文化財保護法　と　文化財保護法（埋蔵文化財）　⇒　埋蔵文化財へ
- 景観法　と　景観地区　⇒　景観法へ
- 東日本震災復興特　と　東日本大震災復興特別区域法　⇒　東日本震災復興特　へ
- 農地法　と　農地法届出要　⇒　農地法　へ

#### 法規制のうち、出現回数が1,2回のものがある。⇒　出現階数が４以上のものに絞ると次のように。本当に絞るかはRandomForestの重要度を見てから決めることにする。
- 公拡法 あわせて１９回
- 区画整理法 4回
- 国土法　11回
- 埋蔵文化財 75+2+39回
- 景観法 549+23回
- 東日本震災復興特 7+1回
- 河川法 10回
- 自然公園法 9回
- 航空法 19回
- 農地法 270+15回
- 風致地区 １９回

In [32]:
# ステップ１：　hokaisei1〜4に対して法規制の名称の統一をする 
# ステップ２：対象とする法規制のダミー変数列を作る
# ステップ3：ダミー変数化する

def put_name(col, name):
    if ( type(name) != float ):
        col.add(name)

# ステップ１：
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')
df = train_genba.copy()
kisei_map = {'公有地拡大推進法':'公拡法', '文化財保護法':'埋蔵文化財', '文化財保護法（埋蔵文化財）':'埋蔵文化財', '景観地区':'景観法','東日本大震災復興特別区域法':'東日本震災復興特', '農地法届出要':'農地法　'}
#kisei_map = {'公有地拡大推進法':'公拡法', '文化財保護法':'埋蔵文化財', '文化財保護法（埋蔵文化財）　':'埋蔵文化財','景観地区':'景観法','東日本大震災復興特別区域法':'東日本震災復興特'}

df['hokakisei1']=df['hokakisei1'].replace(kisei_map)
df['hokakisei2']=df['hokakisei2'].replace(kisei_map)
df['hokakisei3']=df['hokakisei3'].replace(kisei_map)
df['hokakisei4']=df['hokakisei4'].replace(kisei_map)

cols = set()
for i in range(len(train_genba)):
    put_name(cols, df.at[i,'hokakisei1'])
    put_name(cols, df.at[i,'hokakisei2'])
    put_name(cols, df.at[i,'hokakisei3'])
    put_name(cols, df.at[i,'hokakisei4'])

# ステップ２：
kisei_dict = {}
for x in cols:
    kisei_dict[x] = np.zeros(len(df), dtype=np.int)

# ステップ３：
for i in range(len(df)):
    name = df.at[i,'hokakisei1']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = df.at[i,'hokakisei2']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = df.at[i,'hokakisei3']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = df.at[i,'hokakisei4']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1

# 元のDataFrameとダミー変数化したものを連結
new_genba = pd.concat([df, pd.DataFrame(kisei_dict)], axis=1)
new_genba.to_csv("data/temp.csv")





In [20]:
import math

In [2]:
cols = set()
for i in range(len(train_genba)):
    name = train_genba.at[i,'hokakisei1']
    if ( type(name) != float):
        cols.add(name)
    name = train_genba.at[i,'hokakisei2']
    if ( type(name) != float):
        cols.add(name)
    name = train_genba.at[i,'hokakisei3']
    if ( type(name) != float):
        cols.add(name)
    name = train_genba.at[i,'hokakisei4']
    if ( type(name) != float):
        cols.add(name)

kisei_dict = {}
for x in cols:
    kisei_dict[x] = np.zeros(len(train_genba),dtype=np.int)
    
for i in range(len(train_genba)):
    name = train_genba.at[i,'hokakisei1']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = train_genba.at[i,'hokakisei2']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = train_genba.at[i,'hokakisei3']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = train_genba.at[i,'hokakisei4']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1

kisei_df = pd.DataFrame(kisei_dict)
kisei_df.head()

,43条許可,55条許可,下水道法,公拡法,公有地拡大推進法,区画整理法,国土法,土壌汚染対策法,埋蔵文化財,外壁後退,...,河川法,特定空港周辺特別措置法,自然公園法,航空法,農地法,農地法届出要,都市再生特別措置法,風致地区,高さ最高限度有,７６条申請
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
 train_genba.tail()

,pj_no,jukyo,chiseki_js_hb,chiseki_kb_hb,yoto1,yoto2,kempei1,kempei2,yoseki1,yoseki2,...,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2
2776,2776,埼玉県越谷市大字下間久里,110.22,109.94,第一種中高層住居専用地域,NaN,60,0.0,200,0.0,...,徒歩,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2777,2777,埼玉県さいたま市岩槻区加倉,418.26,418.26,第一種低層住居専用地域,NaN,50,0.0,80,0.0,...,徒歩,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2778,2778,埼玉県久喜市栗橋中央,429.40,429.40,第一種住居地域,NaN,60,0.0,200,0.0,...,徒歩,8,NaN,NaN,東武日光線,栗橋,徒歩,8.0,NaN,NaN
2779,2779,埼玉県鴻巣市北新宿,917.32,917.32,第一種中高層住居専用地域,NaN,60,0.0,150,0.0,...,徒歩,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2780,2780,埼玉県川口市領家,617.38,616.71,準工業地域,NaN,60,0.0,200,0.0,...,バス,18,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
new_tgenba = pd.concat([train_genba, kisei_df], axis=1)

In [4]:
new_tgenba.to_csv("data/temp.csv")

In [11]:
pd.DataFrame(kisei_dict).to_csv("data/temp.csv")

##### 6/22の作業

In [ ]:
df = pd.DataFrame()
df['pj']=prod_goto['pj_no']

In [ ]:
df['tc_mseki'] = prod_goto['tc_mseki']

In [ ]:
df[['tt_mseki', 'fukuin']]=prod_goto[['tt_mseki','fukuin']]

ここからモデル作成と予測をするコードを書く

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb 
from xgboost import XGBClassifier 

In [2]:
x_train = pd.read_csv('data/processed_train_goto.csv')
Y_train = pd.DataFrame(x_train['tc_tanka'])

In [3]:
X_train = x_train.drop(['id','pj_no','tc_tanka'],axis=1)
X_train = X_train.values
Y_train = Y_train.values.ravel()

In [ ]:
# XGBoostは時間かかりすぎ
#  XGB = XGBClassifier(n_estimater=500,n_jobs=-1)
# XGB.fit(X_train, Y_train)
#
# ⇒　RandomForestで重要度を見てから、不要な特徴量を落とすことにする
#

In [ ]:
df.head()

#### 6/20の作業

In [ ]:
df = train_genba.copy()
df['gk_yoc_tm'] = df['gk_yoc_tm'].fillna(df['gk_yoc_tm'].mean())

In [ ]:
pd.DataFrame(prod_goto.isnull().sum()).to_csv("data/null_check.csv")

In [ ]:
import numpy as np
import pandas as pd
test_goto = pd.read_csv("data/processed_test_goto.csv")
pd.DataFrame(test_goto.isnull().sum()).to_csv("data/null_check_test.csv")

#### 確認の結果、次のカラムにNULLを含む
#### chiseki_kb_hb,kaoku_hb, mseki_rd_hb, mseki_dp_hb, chiseki_diff

In [ ]:
    # 間口(magutchi)
    # 幅員と同じように補完する。
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    

#### 6/18,19の作業

In [ ]:
df = train_goto.copy()

In [ ]:
genba = pd.read_csv("data/processed_train_genba.csv")

In [ ]:
newdf=df.merge(genba, how="left", on='pj_no')

In [ ]:
newdf.to_csv("data/joined_data.csv")
df = newdf

In [ ]:
df['fukuin'] = df['fukuin'].fillna(0)
for i in range(len(df)):
    if ( df.at[i,'fukuin'] ==0 ) :
        df.at[i,'fukuin'] = df.at[i,'road1_fi']

In [ ]:
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    

In [ ]:
df['setsudo_hi' ] = df['setsudo_hi'].fillna('unknown')
for i in range(len(df)):
    if ( df.at[i,'setsudo_hi']=='unknown'):
        df.at[i.'setsudo_hi']=df.at[i,'road1_hk']

In [ ]:
df.head()

In [ ]:
help(list.append)

In [ ]:
df = pd.read_csv('data/processed_train_goto.csv')

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.loc[146,'fukuin']

In [ ]:
df['fukuin'] = df['fukuin'].fillna(0)

In [ ]:
df['fukuin'].isnull().sum()

#### いつのだかわからないコード

In [ ]:
df = train_genba.copy()

In [ ]:
df[['bus', 'walk']] = df.apply(calculateLogistics, axis=1)

In [ ]:
df.loc[0:20,['bas_toho1','eki_kyori1','teiho1','bus','walk']]

#### 下記は6/16夜の作業

In [ ]:
def calculateLogistics(x):
    if ( x['bas_toho1'] == 'バス' ):
        bus = x['eki_kyori1']
        walk = x['teiho1']
    else:
        bus = 0
        walk = x['eki_kyori1']
    
    return pd.Series([bus, walk])
    

#### 下記は6/16の作業

In [ ]:
df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)

In [ ]:
np.isnan(df.loc[0,'yoto2'])

In [ ]:
type(df.loc[26,'yoto2'])==str

In [ ]:
df.loc[20,'shu_soon']

In [ ]:
df['shu_soon'].map({'○':1, np.NaN:0})

In [ ]:
def select_yoto(x):
    if ( type(x['yoto2']) == str ):
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    else:
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1']
    """
    if ( np.isnan(x['yoto2'])):
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    """
    """"
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    """
    return pd.Series([yoto, kempei, yoseki])


#### 下記は6/15の作業

In [ ]:
df = df.drop('jukyo',axis=1)

In [ ]:
# df['yoto'], df['kempei'],df['yoseki']= df.apply(select_yoto,axis=1)
df[['yoto','kempei','yoseki']] = df.apply(select_yoto,axis=1)

In [ ]:
df.head()

In [ ]:
org_df = df.copy()

In [ ]:
df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)

In [ ]:
df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)

In [ ]:
df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)

In [ ]:
df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']

In [ ]:
a = pd.get_dummies(df[['usui']],drop_first=False)

In [ ]:
df2 = pd.merge(df, a, left_index=True, right_index=True)

In [ ]:
df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)

In [ ]:
df = df.fillna({'tateuri_su': 0})

In [ ]:
df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)

In [ ]:
df['road_num']=df.apply(lambda x: 4 if x['road4_hk']!=np.nan else 3)

In [ ]:
df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)

In [ ]:
import math

In [ ]:
df['road_num']=df.apply(lambda x: 4 if not math.isnan(x['road4_fi']) else 3 if not math.isnan(x['road3_fi']) else 2 if not math.isnan(x['road2_fi']) else 1, axis=1)

In [ ]:
df['toshikuiki']=df['toshikuiki2'].fillna('empty')

In [ ]:
df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)

In [ ]:
df['toshikuiki']

In [ ]:
df.loc[1549,'toshikuiki2']

In [ ]:
def select_yoto(x):
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    return pd.Series([yoto, kempei, yoseki])
